Question: 1

Environment, in general, is defined as anything that falls outside the agent. It communicates with the agent. It determines the reward signal for agent's action and its observation.

For NIM game, specifically, the three piles, qtable and reward giving strategy based on values of reward, learning rate and discount factor constitutes an environment.

Environment also includes rules via which the game is played. As an example, each player has to remove at least one item from the three piles on every turn. Then, the rule that decides the winner is also in the environment.

Question: 2

Agent, in general, is defined as an entity that learns how to make decisions. It interacts with its surroundings by taking an action. In return for taking an action, agent receives a positive or negative feedback from its environment.

In case of NIM game, there are three agents - Random, Guru and Q-Learner.

Random makes random choices throughout without learning hence wins handful number of times against Qlearner and always loses to Guru. Guru makes choies based on solid mathematics and always wins against all opponents. Qlearner learns based on reinforcement learning i.e it learns using the feedback from the environment. It starts by losing games but learns evnetually to start winning.

Question: 3

Reward and Penalty is the feedback obtained by an agent from the environment. Reward signals correct move or desired action. Penalty signals incorrect move or incorrect action.

For the NIM game, value of 100 given to action upon updating of qtable is its reward. If this value was, say -100, then it will be considered as a penalty.


Question: 4

The total number of possible states in the NIM game can be 10 x 10 x 10

Question: 5

The total possible unique actions for 1 player are 10 x 3 = 30

Question: 6
I could not figure out an answer to this question because I didnt know how to implement my possibly incorrect correction (below). Also, I didnt quite understand these concepts. I have asked Prof.Yasin if I can submit this question later.

My suggestion would be to implement Off-Policy TD control instead of equation:3 of the given code.

Q(St, At) <-- Q(St, At) + alpha * [ Rt+1, gamma * max (Q(St+1, a) - Q(St,At)

Guru player cannot be used inside the learnign module because it is based on mathematical rules to make sure it wins all the time. If a Qlearner is made to learn by playng against Guru, it wont learn anything because it will lose all the time.


Question: 6

In [ ]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game()->list:
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(_st:list)->(int,int):
    xored = _st[0] ^ _st[1] ^ _st[2]
    if xored == 0:
        return nim_random(_st)
    for pile in range(3):
        s = _st[pile] ^ xored
        if s <= _st[pile]:
            return _st[pile]-s, pile

# Random Nim player
def nim_random(_st:list)->(int,int):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [ ]:
def nim_qlearner(_st:list)->(int,int):
    global qtable
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action

Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(_a:str, _b:str):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[_a] if side == 'A' else Engines[_b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n:int, _a:str, _b:str)->(int,int):
    from collections import defaultdict
    wins = defaultdict(int)
    for _ in range(_n):
        wins[game(_a, _b)] += 1
    # info
    print(f"{_n} games, {_a:>8s}{wins['A']:5d}  {_b:>8s}{wins['B']:5d}")
    return wins['A'], wins['B']

In [ ]:
# Play games
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru') ;

1000 games,   Random  498    Random  502
1000 games,     Guru 1000    Random    0
1000 games,   Random    5      Guru  995
1000 games,     Guru  941      Guru   59


In [ ]:
qtable, Alpha, Gamma, Reward = None, 1, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n:int):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=np.float32)
    # play _n gameswas
    for _ in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break
            
            elif np.max(qtable[st2[0], st2[1], st2[2]]) >= Reward:
                qtable_update(-Reward, st1, move, pile, np.min(qtable[st2[0], st2[1], st2[2]]))
                # I tried Reward-10 but with only 1000 games to learn from. As per the prof, -Reward
                # should work but it is not working.  
                
            else:    

                qtable_update(Reward, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            
            # Switch sides for play and learning
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r:float, _st1:list, move:int, pile:int, q_future_best:float):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best) # important

In [ ]:
%%time
nim_qlearn(1000000)

CPU times: user 1min 36s, sys: 480 ms, total: 1min 36s
Wall time: 1min 41s


In [ ]:
# Play games
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner')
play_games(1000, 'Random', 'Random') ;

1000 games, Qlearner  999    Random    1
1000 games,   Random    5  Qlearner  995
1000 games,   Random  516    Random  484


In [ ]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
Wins = []
for n in n_train:
    nim_qlearn(n)
    wins_a, wins_b = play_games(1000, 'Qlearner', 'Random')
    Wins += [wins_a/(wins_a+wins_b)]

1000 games, Qlearner  530    Random  470
1000 games, Qlearner  543    Random  457
1000 games, Qlearner  751    Random  249
1000 games, Qlearner  928    Random   72
1000 games, Qlearner  991    Random    9
1000 games, Qlearner  997    Random    3
1000 games, Qlearner  999    Random    1
CPU times: user 14.4 s, sys: 107 ms, total: 14.6 s
Wall time: 14.5 s


In [ ]:
# Check the ratio of wins wrt to size of the reinforcement model training
print(Wins)

[0.53, 0.543, 0.751, 0.928, 0.991, 0.997, 0.999]


In [ ]:
# Function to print the entire set of states
def qtable_log(_fn:str):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            print(s, file=fout)

qtable_log('qtable_debug.txt')